<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-Data" data-toc-modified-id="Read-in-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in Data</a></span></li><li><span><a href="#Generate-Models" data-toc-modified-id="Generate-Models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate Models</a></span><ul class="toc-item"><li><span><a href="#Support-Vector-Machines" data-toc-modified-id="Support-Vector-Machines-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Support Vector Machines</a></span></li><li><span><a href="#Neural-Network" data-toc-modified-id="Neural-Network-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Neural Network</a></span></li></ul></li><li><span><a href="#Run-Models" data-toc-modified-id="Run-Models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run Models</a></span></li></ul></div>

In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import tensorflow as tf

## Read in csv 
tot_df_clean_year = pd.read_csv('model_data.csv', 
                                index_col = ['year', 'ID']).dropna() 
tot_df_clean_year.head() 

lead_rating  rating  IQ_TOTAL_ASSETS  IQ_TOTAL_REV  IQ_NI_CF  \
year ID                                                                   
2001 JPM          -1.0     1.0        -0.945447     -0.002409 -0.000409   
     F            -1.0    -1.0         0.143388     -0.042495 -0.004145   
     CHTR         -1.0    -0.0         1.445774     -0.000547 -0.001261   
     DOV          -0.0    -0.0        -0.958897     -0.013956 -0.014696   
     CCI          -0.0    -0.0         0.054378     -0.000804 -0.003302   

              IQ_AR    IQ_GP      IQ_AP  IQ_TOTAL_LIAB  IQ_TOTAL_DEBT  \
year ID                                                                 
2001 JPM   0.000000 -0.002409  0.004809       0.001093      -0.005029   
     F    -0.005853 -0.008635  0.002457       0.020101       0.018932   
     CHTR  0.000173  0.003984  0.023845       0.007280       0.048779   
     DOV  -0.019740 -0.014525 -0.008472      -0.020854      -0.027863   
     CCI   0.000723 -0.000466 -0.002664       0.049090       0.033564   

           IQ_CASH_FINAN  IQ_TOTAL_EQUITY  IQ_CASH_EQUIV  
year ID                                                   
2001 JPM       -0.009855        -0.001093      -0.026098  
     F         -0.015464        -0.020101       0.004941  
     CHTR       0.004572        -0.007280       0.000449  
     DOV       -0.015357         0.020854       0.010926  
     CCI        0.035338        -0.049090       0.022043

## Read in Data

In [2]:
## Test and train split for time series
train = tot_df_clean_year.head(
    int(tot_df_clean_year.shape[0] * 0.8 ) 
) 

test = tot_df_clean_year.tail(
    int(tot_df_clean_year.shape[0] * 0.2) 
)

## Split into x and y 
def SplitData(df): 
    y = df.lead_rating 
    x = df.drop('lead_rating', axis = 1) 
    
    ## Standardize entire data 
    x = (x  - x.mean()) / x.std() 
    return x, y 

train_x, train_y = SplitData(train) 
test_x, test_y = SplitData(test) 

## Generate Models 
### Support Vector Machines

In [3]:
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


def SVM_Fit(train_x, train_y, kernel,
            params = [10**x for x in np.arange(-1,3,0.9)]): 
    '''Fit the SVM Machine given the kernel type, parameters, 
    data''' 
    
    if kernel == 'linear': 
        parameters = {'C': params} 
    else: 
        parameters = {'C': params, 
                     'gamma': params} 
    
    cv = TimeSeriesSplit(n_splits = 5) 
    
    model = GridSearchCV(estimator = SVC(kernel = kernel), 
                        param_grid = parameters, 
                        cv = cv, 
                        verbose = 1) 
    
    model.fit(train_x, train_y) 
    return model

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

def RandomForestsModel(train_x, train_y): 
    '''Random Forest Model'''
    rf = RandomForestClassifier(random_state = 200) 
    rf.get_params()
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
    
    cv = TimeSeriesSplit(n_splits = 5) 
    
    rf_random = RandomizedSearchCV(estimator = rf, 
                              param_distributions = random_grid, 
                              n_iter = 100, cv = cv, verbose = 2, 
                              random_state = 200, n_jobs = -1)
    
    rf_random.fit(train_x, train_y) 
    return rf_random 

In [5]:
def Predict(fitted_model, test_x, test_y, 
           name):
    '''Prediction Accuracy'''
    prediction = fitted_model.predict(test_x) 
    score = accuracy_score(prediction, test_y) 
    prediction = pd.DataFrame({'prediction_{}'.format(name): prediction})
    print('The {} Model Score is: {}'.format(name, score)) 
    return prediction, score    

### Neural Network

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])

model.stop_training = True

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Run Models

In [8]:
# Category range must go from 0 to 5 instead of -2 to 2
y_train = train_y +2 
y_test = test_y +2

model.fit(train_x.to_numpy(), 
          y_train.to_numpy(),
          epochs=1000)


Train on 1450 samples
Epoch 1/1000
1450/1450 [==============================] - 1s 466us/sample - loss: 1.4322 - accuracy: 0.3745
Epoch 2/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 1.1243 - accuracy: 0.5552
Epoch 3/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 1.0043 - accuracy: 0.5924
Epoch 4/1000
1450/1450 [==============================] - 0s 61us/sample - loss: 0.9399 - accuracy: 0.6297
Epoch 5/1000
1450/1450 [==============================] - 0s 56us/sample - loss: 0.8999 - accuracy: 0.6379
Epoch 6/1000
1450/1450 [==============================] - 0s 58us/sample - loss: 0.8876 - accuracy: 0.6572
Epoch 7/1000
1450/1450 [==============================] - 0s 60us/sample - loss: 0.8751 - accuracy: 0.6510
Epoch 8/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 0.8516 - accuracy: 0.6586
Epoch 9/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 0.8455 - accuracy: 0.6641
Epoch 10/1000


1450/1450 [==============================] - 0s 56us/sample - loss: 0.7529 - accuracy: 0.6793
Epoch 77/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7348 - accuracy: 0.6966
Epoch 78/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.7468 - accuracy: 0.6834
Epoch 79/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7533 - accuracy: 0.6883
Epoch 80/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7587 - accuracy: 0.6876
Epoch 81/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7558 - accuracy: 0.6862
Epoch 82/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7512 - accuracy: 0.6876
Epoch 83/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7512 - accuracy: 0.6931
Epoch 84/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7479 - accuracy: 0.6869
Epoch 85/1000
1450/1450 [=================

1450/1450 [==============================] - 0s 54us/sample - loss: 0.7184 - accuracy: 0.6979
Epoch 152/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7195 - accuracy: 0.6903
Epoch 153/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7234 - accuracy: 0.7014
Epoch 154/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7258 - accuracy: 0.6986
Epoch 155/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7199 - accuracy: 0.6883
Epoch 156/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7082 - accuracy: 0.6979
Epoch 157/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7204 - accuracy: 0.7007
Epoch 158/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.7263 - accuracy: 0.7021
Epoch 159/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7147 - accuracy: 0.6979
Epoch 160/1000
1450/1450 [========

1450/1450 [==============================] - 0s 65us/sample - loss: 0.6941 - accuracy: 0.7145
Epoch 227/1000
1450/1450 [==============================] - 0s 87us/sample - loss: 0.7071 - accuracy: 0.7000
Epoch 228/1000
1450/1450 [==============================] - 0s 87us/sample - loss: 0.7015 - accuracy: 0.6993
Epoch 229/1000
1450/1450 [==============================] - 0s 80us/sample - loss: 0.7136 - accuracy: 0.6952
Epoch 230/1000
1450/1450 [==============================] - 0s 78us/sample - loss: 0.7151 - accuracy: 0.6959
Epoch 231/1000
1450/1450 [==============================] - 0s 80us/sample - loss: 0.7071 - accuracy: 0.7069
Epoch 232/1000
1450/1450 [==============================] - 0s 83us/sample - loss: 0.7101 - accuracy: 0.6931
Epoch 233/1000
1450/1450 [==============================] - 0s 81us/sample - loss: 0.7160 - accuracy: 0.7090
Epoch 234/1000
1450/1450 [==============================] - 0s 79us/sample - loss: 0.7070 - accuracy: 0.7000
Epoch 235/1000
1450/1450 [========

1450/1450 [==============================] - 0s 56us/sample - loss: 0.6960 - accuracy: 0.7014
Epoch 302/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6894 - accuracy: 0.7090
Epoch 303/1000
1450/1450 [==============================] - 0s 56us/sample - loss: 0.6943 - accuracy: 0.7076
Epoch 304/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.6890 - accuracy: 0.7090
Epoch 305/1000
1450/1450 [==============================] - 0s 64us/sample - loss: 0.6924 - accuracy: 0.7117
Epoch 306/1000
1450/1450 [==============================] - 0s 66us/sample - loss: 0.6913 - accuracy: 0.7055
Epoch 307/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7032 - accuracy: 0.7021
Epoch 308/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.7100 - accuracy: 0.7000
Epoch 309/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 0.7090 - accuracy: 0.7034
Epoch 310/1000
1450/1450 [========

...

1450/1450 [==============================] - 0s 66us/sample - loss: 0.6351 - accuracy: 0.7317
Epoch 677/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6531 - accuracy: 0.7221
Epoch 678/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6779 - accuracy: 0.7179
Epoch 679/1000
1450/1450 [==============================] - 0s 87us/sample - loss: 0.6608 - accuracy: 0.7241
Epoch 680/1000
1450/1450 [==============================] - 0s 61us/sample - loss: 0.6509 - accuracy: 0.7317
Epoch 681/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.6564 - accuracy: 0.7255
Epoch 682/1000
1450/1450 [==============================] - 0s 56us/sample - loss: 0.6595 - accuracy: 0.7069
Epoch 683/1000
1450/1450 [==============================] - 0s 65us/sample - loss: 0.6650 - accuracy: 0.7234
Epoch 684/1000
1450/1450 [==============================] - 0s 63us/sample - loss: 0.6665 - accuracy: 0.7186
Epoch 685/1000
1450/1450 [========

1450/1450 [==============================] - 0s 54us/sample - loss: 0.6705 - accuracy: 0.7159
Epoch 752/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6456 - accuracy: 0.7145
Epoch 753/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 0.6672 - accuracy: 0.7248
Epoch 754/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6632 - accuracy: 0.7310
Epoch 755/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6578 - accuracy: 0.7241
Epoch 756/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6545 - accuracy: 0.7276
Epoch 757/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6527 - accuracy: 0.7214
Epoch 758/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.6559 - accuracy: 0.7200
Epoch 759/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.6460 - accuracy: 0.7317
Epoch 760/1000
1450/1450 [========

1450/1450 [==============================] - 0s 61us/sample - loss: 0.6462 - accuracy: 0.7303
Epoch 827/1000
1450/1450 [==============================] - 0s 60us/sample - loss: 0.6525 - accuracy: 0.7241
Epoch 828/1000
1450/1450 [==============================] - 0s 60us/sample - loss: 0.6626 - accuracy: 0.7145
Epoch 829/1000
1450/1450 [==============================] - 0s 59us/sample - loss: 0.6431 - accuracy: 0.7234
Epoch 830/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6477 - accuracy: 0.7276
Epoch 831/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6507 - accuracy: 0.7297
Epoch 832/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6530 - accuracy: 0.7207
Epoch 833/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6525 - accuracy: 0.7338
Epoch 834/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6423 - accuracy: 0.7255
Epoch 835/1000
1450/1450 [========

1450/1450 [==============================] - 0s 54us/sample - loss: 0.6278 - accuracy: 0.7414
Epoch 902/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6778 - accuracy: 0.7097
Epoch 903/1000
1450/1450 [==============================] - 0s 59us/sample - loss: 0.6420 - accuracy: 0.7310
Epoch 904/1000
1450/1450 [==============================] - 0s 54us/sample - loss: 0.6738 - accuracy: 0.7152
Epoch 905/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6501 - accuracy: 0.7228
Epoch 906/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6323 - accuracy: 0.7331
Epoch 907/1000
1450/1450 [==============================] - 0s 53us/sample - loss: 0.6535 - accuracy: 0.7310
Epoch 908/1000
1450/1450 [==============================] - 0s 55us/sample - loss: 0.6478 - accuracy: 0.7241
Epoch 909/1000
1450/1450 [==============================] - 0s 57us/sample - loss: 0.6461 - accuracy: 0.7193
Epoch 910/1000
1450/1450 [========

1450/1450 [==============================] - 0s 58us/sample - loss: 0.6306 - accuracy: 0.7338
Epoch 977/1000
1450/1450 [==============================] - 0s 58us/sample - loss: 0.6511 - accuracy: 0.7386
Epoch 978/1000
1450/1450 [==============================] - 0s 56us/sample - loss: 0.6550 - accuracy: 0.7193
Epoch 979/1000
1450/1450 [==============================] - 0s 64us/sample - loss: 0.6359 - accuracy: 0.7386
Epoch 980/1000
1450/1450 [==============================] - 0s 63us/sample - loss: 0.6313 - accuracy: 0.7414
Epoch 981/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6639 - accuracy: 0.7262
Epoch 982/1000
1450/1450 [==============================] - 0s 62us/sample - loss: 0.6513 - accuracy: 0.7262
Epoch 983/1000
1450/1450 [==============================] - 0s 70us/sample - loss: 0.6505 - accuracy: 0.7297
Epoch 984/1000
1450/1450 [==============================] - 0s 91us/sample - loss: 0.6400 - accuracy: 0.7366
Epoch 985/1000
1450/1450 [========

In [9]:
results = model.evaluate(test_x.to_numpy(),  y_test.to_numpy(), verbose=2)
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = np.argmax(probability_model.predict(test_x.to_numpy()),axis=1)
confusion_matrix = tf.math.confusion_matrix(y_test, predictions)
# np.argmax(predictions,axis=1)
print("Tensorflow",confusion_matrix.numpy(),sep="\n")

362/1 - 0s - loss: 0.9575 - accuracy: 0.6160
Tensorflow
[[  0   0   0   0]
 [  0  37  16   4]
 [  0  25  79  21]
 [  0  11  62 107]]


In [ ]:
## Make predictions
sigmoid = SVM_Fit(train_x, train_y, 'sigmoid') 
rbf = SVM_Fit(train_x, train_y, 'rbf') 
linear = SVM_Fit(train_x, train_y,'linear')
#poly = SVM_Fit(train_x, train_y, 'poly') 
poly = SVC(kernel = 'poly').fit(train_x, train_y)

rf = RandomForestsModel(train_x, train_y) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [ ]:
## SVM Score
sigmoid_predict, sigmoid_score = Predict(sigmoid, test_x, test_y, 'sigmoid') 
lin_predict, lin_score = Predict(linear, test_x, test_y, 'linear') 
poly_predict, poly_score = Predict(poly, test_x, test_y, 'poly') 
rbf_predict, rbf_score = Predict(rbf, test_x, test_y, 'radial')

## TF Score 
tf_score = accuracy_score(predictions, y_test.to_numpy()) 

## Random Forest Score
random_predict, random_score = Predict(rf, test_x, test_y, 
                                      'Random Forest')

In [ ]:
from sklearn.metrics import confusion_matrix
print("Sigmoid",confusion_matrix(test_y,sigmoid_predict,normalize="true"),sep="\n")
print("Linear",confusion_matrix(test_y,lin_predict,normalize="true"),sep="\n")
print("Poly",confusion_matrix(test_y,poly_predict,normalize="true"),sep="\n")
print("Radial",confusion_matrix(test_y,rbf_predict,normalize="true"),sep="\n")
print("Random Forest",confusion_matrix(test_y,random_predict,normalize="true"),sep="\n")

In [ ]:
fig = go.Figure() 
model_names = ['Sigmoid SVM', 'Radial SVM', 'Linear SVM', 'Polynomial SVM', 'Neural Network']

model_accuracy = [sigmoid_score, rbf_score, lin_score, poly_score, tf_score]

fig.add_trace(go.Bar(x = model_names, 
                    y = model_accuracy, 
                    text = model_accuracy, 
                    textposition = 'auto'))
fig.update_layout(title = 'Model Accuracy Scores Numeric Prediction')

fig.update_yaxes(title_text = 'Accuracy Score') 
fig.update_xaxes(title_text = "Model")
fig.show()